### Entanglement-based Quantum Key Distribution in a 3x3 grid topology using Netsquid
#### This implementation uses the Ekert Protocol for Quantum Key Distribution in a 3x3 grid Quantum Network. After the Quantum Network is defined, the simulation starts dynamic protocols depending on the connection requirements.
_refs_: 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater_chain.html 
* https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater.html 
* Evan Sutcliffe, Matty J. Hoban & Alejandra Beghelli - Multipath Routing for Multipartite State Distribution in Quantum Networks  

_(adapted by D-Cryp7 for Netsquid 1.1.6)_

#### Limitations:
* Routes of two nodes doesn't work yet. The aim of this implementation is for testing the entanglement swapping.
* Some routes doesn't work, i don't know why because i didn't found a public quantum network implementation in Netsquid, so there's no validation yet.
* The quantum network needs to be reset for each traffic because of a ProcessorBusyError on defining the Swap and Correct protocols of each node. Maybe it's necessary to create a quantum processor for each link.

We hope to fix this limitations in future updates.

In [1]:
# Imports

import pandas
import pydynaa
import numpy as np

import netsquid as ns
from netsquid.qubits import ketstates as ks
from netsquid.components import Message, QuantumProcessor, QuantumProgram, PhysicalInstruction
from netsquid.components.models.qerrormodels import DepolarNoiseModel, DephaseNoiseModel, QuantumErrorModel
from netsquid.components.instructions import INSTR_MEASURE_BELL, INSTR_X, INSTR_Z
from netsquid.nodes import Node, Network
from netsquid.protocols import LocalProtocol, NodeProtocol, Signals
from netsquid.util.datacollector import DataCollector
from netsquid.examples.teleportation import EntanglingConnection, ClassicalConnection

#### Methods extracted directly from the reference

In [2]:
class FibreDepolarizeModel(QuantumErrorModel):
    """Custom non-physical error model used to show the effectiveness
    of repeater chains.

    The default values are chosen to make a nice figure,
    and don't represent any physical system.

    Parameters
    ----------
    p_depol_init : float, optional
        Probability of depolarization on entering a fibre.
        Must be between 0 and 1. Default 0.009
    p_depol_length : float, optional
        Probability of depolarization per km of fibre.
        Must be between 0 and 1. Default 0.025

    """

    def __init__(self, p_depol_init=0.009, p_depol_length=0.025):
        super().__init__()
        self.properties['p_depol_init'] = p_depol_init
        self.properties['p_depol_length'] = p_depol_length
        self.required_properties = ['length']

    def error_operation(self, qubits, delta_time=0, **kwargs):
        """Uses the length property to calculate a depolarization probability,
        and applies it to the qubits.

        Parameters
        ----------
        qubits : tuple of :obj:`~netsquid.qubits.qubit.Qubit`
            Qubits to apply noise to.
        delta_time : float, optional
            Time qubits have spent on a component [ns]. Not used.

        """
        for qubit in qubits:
            prob = 1 - (1 - self.properties['p_depol_init']) * np.power(
                10, - kwargs['length']**2 * self.properties['p_depol_length'] / 10)
            ns.qubits.depolarize(qubit, prob=prob)

In [3]:
class SwapCorrectProgram(QuantumProgram):
    """Quantum processor program that applies all swap corrections."""
    default_num_qubits = 1

    def set_corrections(self, x_corr, z_corr):
        self.x_corr = x_corr % 2
        self.z_corr = z_corr % 2

    def program(self):
        q1, = self.get_qubit_indices(1)
        if self.x_corr == 1:
            self.apply(INSTR_X, q1)
        if self.z_corr == 1:
            self.apply(INSTR_Z, q1)
        yield self.run()

In [4]:
def create_qprocessor(name):
    """Factory to create a quantum processor for each node in the repeater chain network.

    Has two memory positions and the physical instructions necessary for teleportation.

    Parameters
    ----------
    name : str
        Name of the quantum processor.

    Returns
    -------
    :class:`~netsquid.components.qprocessor.QuantumProcessor`
        A quantum processor to specification.

    """
    noise_rate = 200
    gate_duration = 1
    gate_noise_model = DephaseNoiseModel(noise_rate)
    mem_noise_model = DepolarNoiseModel(noise_rate)
    physical_instructions = [
        PhysicalInstruction(INSTR_X, duration=gate_duration,
                            quantum_noise_model=gate_noise_model),
        PhysicalInstruction(INSTR_Z, duration=gate_duration,
                            quantum_noise_model=gate_noise_model),
        PhysicalInstruction(INSTR_MEASURE_BELL, duration=gate_duration),
    ]
    qproc = QuantumProcessor(name, num_positions=2, fallback_to_nonphysical=False,
                             mem_noise_models=[mem_noise_model] * 2,
                             phys_instructions=physical_instructions)
    return qproc

#### Methods created/modified by D-Cryp7

In [5]:
def get_neighbours(network, node, n):
    '''
        Get neighbours of each node in a nxn grid network
    '''
    neighbours = []
    nodes = network.nodes
    pos = eval(node.name)
    i, j = pos[0], pos[1]
    if i + 1 < n:
        neighbours.append(nodes[str((i + 1, j))])
    if i - 1 >= 0:
        neighbours.append(nodes[str((i - 1, j))])
    if j + 1 < n:
        neighbours.append(nodes[str((i, j + 1))])
    if j - 1 >= 0:
        neighbours.append(nodes[str((i, j - 1))])
    return neighbours

In [6]:
def setup_datacollector(network, protocol, path):
    """Setup the datacollector to calculate the fidelity
    when the CorrectionProtocol has finished.

    Parameters
    ----------
    network : :class:`~netsquid.nodes.network.Network`
        Repeater chain network to put protocols on.

    protocol : :class:`~netsquid.protocols.protocol.Protocol`
        Protocol holding all subprotocols used in the network.

    Returns
    -------
    :class:`~netsquid.util.datacollector.DataCollector`
        Datacollector recording fidelity data.

    """
    # Ensure nodes are ordered in the chain:
    # nodes = [network.nodes[name] for name in sorted(network.nodes.keys())]
    nodes = [network.nodes[str(name)] for name in path]

    def calc_fidelity(evexpr):
        qubit_a, = nodes[0].qmemory.peek([0])
        qubit_b, = nodes[-1].qmemory.peek([1])
        fidelity = ns.qubits.fidelity([qubit_a, qubit_b], ks.b00, squared=True)
        return {"fidelity": fidelity}

    dc = DataCollector(calc_fidelity, include_entity_name=False)
    dc.collect_on(pydynaa.EventExpression(source=protocol.subprotocols['CorrectProtocol'],
                                          event_type=Signals.SUCCESS.value))
    return dc

In [7]:
class SwapProtocol(NodeProtocol):
    """Perform Swap on a repeater node.

    Parameters
    ----------
    node : :class:`~netsquid.nodes.node.Node` or None, optional
        Node this protocol runs on.
    name : str
        Name of this protocol.

    """

    def __init__(self, node, name, ccon_R):
        super().__init__(node, name)
        self._qmem_input_port_l = self.node.qmemory.ports["qin1"]
        self._qmem_input_port_r = self.node.qmemory.ports["qin0"]
        self._program = QuantumProgram(num_qubits=2)
        self.ccon_R = ccon_R
        q1, q2 = self._program.get_qubit_indices(num_qubits=2)
        self._program.apply(INSTR_MEASURE_BELL, [q1, q2], output_key="m", inplace=False)

    def run(self):
        while True:
            yield (self.await_port_input(self._qmem_input_port_l) &
                   self.await_port_input(self._qmem_input_port_r))
            # Perform Bell measurement
            yield self.node.qmemory.execute_program(self._program, qubit_mapping=[1, 0])
            m, = self._program.output["m"]
            # Send result to right node on end
            self.node.ports[self.ccon_R].tx_output(Message(m))
            # print(f"Message swapped from {self.ccon_R}: {m}")

In [8]:
class CorrectProtocol(NodeProtocol):
    """Perform corrections for a swap on an end-node.

    Parameters
    ----------
    node : :class:`~netsquid.nodes.node.Node` or None, optional
        Node this protocol runs on.
    num_nodes : int
        Number of nodes in the repeater chain network.

    """

    def __init__(self, node, num_nodes, ccon_L):
        super().__init__(node, "CorrectProtocol")
        self.num_nodes = num_nodes
        self._x_corr = 0
        self._z_corr = 0
        self._program = SwapCorrectProgram()
        self._counter = 0
        self.ccon_L = ccon_L

    def run(self):
        while True:
            yield self.await_port_input(self.node.ports[self.ccon_L])
            message = self.node.ports[self.ccon_L].rx_input()
            if message is None or len(message.items) != 1:
                continue
            m = message.items[0]
            if m == ks.BellIndex.B01 or m == ks.BellIndex.B11:
                self._x_corr += 1
            if m == ks.BellIndex.B10 or m == ks.BellIndex.B11:
                self._z_corr += 1
            self._counter += 1
            if self._counter == self.num_nodes - 2:
                if self._x_corr or self._z_corr:
                    self._program.set_corrections(self._x_corr, self._z_corr)
                    yield self.node.qmemory.execute_program(self._program, qubit_mapping=[1])
                self.send_signal(Signals.SUCCESS)
                self._x_corr = 0
                self._z_corr = 0
                self._counter = 0
                # print(f"Message recieved and corrected from {self.ccon_L}: {m}")

In [9]:
def network_setup(n, node_distance, source_frequency):
    
    """
        3x3 grid Quantum Network setup.
        - based on Multipath Routing for Multipartite State Distribution in Quantum Networks
        - each node can do Entanglement Swapping
        
        Parameters:
        - node_distance (float): Distance between nodes [km]
        - source_frecuency (float): Frequency at which the sources create entangled qubits [Hz]
        
        Returns:
        - class:`~netsquid.nodes.network.Network`
          Network component with all nodes and connections as subcomponents.
          
        ref: https://docs.netsquid.org/latest-release/learn_examples/learn.examples.repeater_chain.html
    """
    network = Network("3x3 grid Quantum Network")
    # Create nodes with quantum processors
    nodes = []
    for i in range(n):
        for j in range(n):
            nodes.append(Node(f"{i,j}", qmemory = create_qprocessor(f"qproc_{i,j}")))
    network.add_nodes(nodes)
    # Create quantum and classical connections:
    for i in range(len(nodes)):
        current_node = nodes[i]
        # print(current_node.qmemory.ports)
        neighbours = get_neighbours(network, current_node, 3)
        # print("Current node neighbours: ", neighbours)
        for near in neighbours:
            # print("Establishing a connection: ", current_node.name, "->", near.name)
            # Create quantum connection
            try:       
                qconn = EntanglingConnection(name=f"qconn_{current_node.name}<->{near.name}", length=node_distance,
                                             source_frequency=source_frequency)
                # Add a noise model which depolarizes the qubits exponentially
                # depending on the connection length
                for channel_name in ['qchannel_C2A', 'qchannel_C2B']:
                    qconn.subcomponents[channel_name].models['quantum_noise_model'] =\
                        FibreDepolarizeModel()
                
                port_name, port_r_name = network.add_connection(
                    current_node, near, connection = qconn, label="quantum")
                # print(port_name, port_r_name)
                # Forward qconn directly to quantum memories for right and left inputs:
                current_node.ports[port_name].forward_input(current_node.qmemory.ports["qin0"])  # R input
                near.ports[port_r_name].forward_input(
                    near.qmemory.ports["qin1"])  # L input
            except:
                pass
            # Create classical connection
            cconn = ClassicalConnection(name=f"cconn_{current_node.name}-{near.name}", length=node_distance)
            port_name, port_r_name = network.add_connection(
                current_node, near, connection=cconn, label="classical",
                port_name_node1 = f"ccon_R_{current_node.name}-{near.name}", port_name_node2 = f"ccon_L_{current_node.name}-{near.name}")
            # Forward cconn to right most node
            if f"ccon_L_{current_node.name}-{near.name}" in current_node.ports:
                current_node.ports[f"ccon_L_{current_node.name}-{near.name}"].bind_input_handler(
                    lambda message, _node = current_node: _node.ports[f"ccon_R_{current_node.name}-{near.name}"].tx_output(message))
    return network

In [10]:
def setup_repeater_protocol(network, path):
    """Setup repeater protocol on repeater chain network.

    Parameters
    ----------
    network : :class:`~netsquid.nodes.network.Network`
        Repeater chain network to put protocols on.

    Returns
    -------
    :class:`~netsquid.protocols.protocol.Protocol`
        Protocol holding all subprotocols used in the network.

    """
    nodes = [network.nodes[str(name)] for name in path]
    protocol = LocalProtocol(nodes = network.nodes)
    # Add SwapProtocol to all repeater nodes. Note: we use unique names,
    # since the subprotocols would otherwise overwrite each other in the main protocol.
    nodes = [network.nodes[str(name)] for name in path]
    for node in nodes[1:-1]:
        index = path.index(eval(node.name))
        ccon_R = f"ccon_R_{node.name}-{nodes[index + 1].name}"
        subprotocol = SwapProtocol(node = node, name = f"Swap_{node.name}", ccon_R = ccon_R) # especificar el puerto ccon_R
        protocol.add_subprotocol(subprotocol)
    # Add CorrectProtocol to Bob
    ccon_L = f"ccon_L_{nodes[-2].name}-{nodes[-1].name}"
    subprotocol = CorrectProtocol(nodes[-1], len(nodes), ccon_L) # especificar el puerto ccon_L
    protocol.add_subprotocol(subprotocol)
    return protocol

In [11]:
def run_simulation(num_nodes, node_distance, num_iters, traffic):
    """Run the simulation experiment and return the collected data.

    Parameters
    ----------
    num_nodes : int, optional
        Number of nodes in the path
    node_distance : float, optional
        Distance between nodes, larger than 0. Default 20 [km].
    num_iters : int, optional
        Number of simulation runs. Default 100.

    Returns
    -------
    :class:`pandas.DataFrame`
        Dataframe with recorded fidelity data.

    """
    ns.sim_reset()
    est_runtime = (0.5 + num_nodes - 1) * node_distance * 5e3
    network = network_setup(3, node_distance = node_distance,
                            source_frequency = 1e9 / est_runtime)
    '''
    for conn in network.connections:
        print(conn)
    '''
    data_collectors = []
    for path in traffic:
        protocol = setup_repeater_protocol(network, path)
        dc = setup_datacollector(network, protocol, path)
        data_collectors.append(dc)
        protocol.start()
    ns.sim_run(est_runtime * num_iters)
    return data_collectors, network

In [12]:
def run_simulation(num_nodes, node_distance, num_iters, path):
    """Run the simulation experiment and return the collected data.

    Parameters
    ----------
    num_nodes : int, optional
        Number of nodes in the path
    node_distance : float, optional
        Distance between nodes, larger than 0. Default 20 [km].
    num_iters : int, optional
        Number of simulation runs. Default 100.

    Returns
    -------
    :class:`pandas.DataFrame`
        Dataframe with recorded fidelity data.

    """
    ns.sim_reset()
    est_runtime = (0.5 + num_nodes - 1) * node_distance * 5e3
    network = network_setup(3, node_distance = node_distance,
                            source_frequency = 1e9 / est_runtime)
    '''
    for conn in network.connections:
        print(conn)
    '''
    protocol = setup_repeater_protocol(network, path)
    dc = setup_datacollector(network, protocol, path)
    protocol.start()
    ns.sim_run(est_runtime * num_iters)
    return dc.dataframe

In [13]:
# Traffic example
traffic = {
    "src": [(0, 0), (1, 0), (1, 1), (0, 2)],
    "dst": [(1, 1), (2, 1), (2, 0), (2, 0)],
    "path": [[(0, 0), (1, 0), (1, 1)], 
             [(1, 0), (1, 1), (2, 1)], 
             [(1, 1), (2, 1), (2, 0)],
             [(0, 2), (1, 2), (2, 2), (2, 1), (2, 0)]]
}

In [14]:
# Simulation
if __name__ == "__main__":
    # Default values
    num_nodes = 5 # maximum
    node_distance = 20
    num_iters = 100
    est_runtime = (0.5 + num_nodes - 1) * node_distance * 5e3
    network = network_setup(3, node_distance = node_distance,
                            source_frequency = 1e9 / est_runtime)
    results = {
        "connection": [],
        "df": []
    }
    for path in traffic["path"]:
        df = run_simulation(len(path), node_distance, num_iters, path)
        fid = df[df["fidelity"] == 1]
        results["connection"].append(path)
        results["df"].append(df)

/home/kali/anaconda3/envs/sage/lib/python3.8/site-packages/netsquid/util/datacollector.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self._dataframe = self._dataframe.append(self._data_buffer, ignore_index=True, sort=False)
/home/kali/anaconda3/envs/sage/lib/python3.8/site-packages/netsquid/util/datacollector.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self._dataframe = self._dataframe.append(self._data_buffer, ignore_index=True, sort=False)
/home/kali/anaconda3/envs/sage/lib/python3.8/site-packages/netsquid/util/datacollector.py:242: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self._dataframe = self._dataframe.append(self._data_buffer, ignore_index=True, sort=False)
/home/kali/anaconda3/envs/sage/lib/py

In [15]:
# Results for each path
for i in range(len(results["connection"])):
    print(results["connection"][i], "\n", results["df"][i])

[(0, 0), (1, 0), (1, 1)] 
     time_stamp  fidelity
0     150001.0      0.25
1     400001.0      0.25
2     650002.0      0.00
3     900003.0      0.25
4    1150002.0      0.25
..         ...       ...
95  23900002.0      0.25
96  24150002.0      0.25
97  24400003.0      0.25
98  24650002.0      1.00
99  24900002.0      0.25

[100 rows x 2 columns]
[(1, 0), (1, 1), (2, 1)] 
     time_stamp  fidelity
0     150003.0      0.50
1     400002.0      0.00
2     650002.0      1.00
3     900002.0      0.25
4    1150002.0      0.50
..         ...       ...
95  23900002.0      0.25
96  24150002.0      0.00
97  24400002.0      0.00
98  24650002.0      1.00
99  24900003.0      0.25

[100 rows x 2 columns]
[(1, 1), (2, 1), (2, 0)] 
     time_stamp  fidelity
0     150003.0      0.50
1     400002.0      0.25
2     650002.0      0.00
3     900003.0      0.25
4    1150002.0      0.25
..         ...       ...
95  23900002.0      0.25
96  24150001.0      0.00
97  24400001.0      0.25
98  24650002.0      0

In [17]:
network.get_connected_ports("(2, 2)", "(2, 1)", "classical") # for research :p

('ccon_R_(2, 2)-(2, 1)', 'ccon_L_(2, 2)-(2, 1)')